# Estrazione Dati Comune - Web Scraping & Document Retrieval

## Overview
Questo notebook estrae i dati mancanti dai CSV dei comuni italiani attraverso:
- Web scraping dei siti comunali
- Download e analisi di documenti PDF
- Retrieval basato su TF-IDF
- Query intelligenti con template multipli

## Features
- Categorizzazione automatica delle celle mancanti
- Generazione di 8-20 query per cella
- Estrazione testo da PDF e HTML
- Ranking dei documenti con TF-IDF
- Audit completo di query e sorgenti

**Autore**: Sistema automatizzato di estrazione dati

**Versione**: 1.0

In [ ]:
!pip install -q pandas requests beautifulsoup4 trafilatura scikit-learn rapidfuzz pdfplumber tqdm lxml

In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import trafilatura
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from rapidfuzz import fuzz
import pdfplumber
from tqdm import tqdm
import warnings
import re
import os
import json
import numpy as np
from urllib.parse import urljoin, urlparse
from pathlib import Path
from datetime import datetime
import time

warnings.filterwarnings('ignore')

print("Libraries imported successfully")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

print("Google Drive mounted successfully")

In [ ]:
# User inputs
base_url = input("Enter base URL of the municipality website (e.g., https://www.comune.vigone.to.it/): ")
if not base_url.endswith('/'):  # Validation: add '/' if missing
    base_url += '/'
    print(f"URL corrected to: {base_url}")

nome_comune = input("Enter municipality name (default: 'Comune'): ") or "Comune"

# Hardcoded years to fill
YEARS_TO_FILL = [2023, 2024]

# Allow external official sources
allow_external_input = input("Allow external official sources (ISTAT, MEF, ISPRA)? (True/False, default: False): ") or "False"
ALLOW_EXTERNAL_OFFICIAL = allow_external_input.lower() == 'true'

# Extract domain from base_url
parsed_url = urlparse(base_url)
DOMAIN = parsed_url.netloc

# Directory paths
INPUT_DIR = "/content/drive/MyDrive/vigone_csv/"
OUTPUT_DIR = "/content/drive/MyDrive/vigone_output/"

# Create output directory
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Print configuration
print("\n=== CONFIGURATION ===")
print(f"Base URL: {base_url}")
print(f"Domain: {DOMAIN}")
print(f"Municipality: {nome_comune}")
print(f"Years to fill: {YEARS_TO_FILL}")
print(f"Allow external sources: {ALLOW_EXTERNAL_OFFICIAL}")
print(f"Input directory: {INPUT_DIR}")
print(f"Output directory: {OUTPUT_DIR}")
print("=====================\n")

In [ ]:
# QueryBuilder Module - Part 1: Synonyms and Query TemplatesSYNONYMS = {    'delibere': ['deliberazioni', 'delibera', 'decreto', 'atto'],    'consiglio': ['cc', 'consiglio comunale', 'council'],    'giunta': ['gc', 'giunta comunale', 'giunta municipale'],    'personale': ['dipendenti', 'organico', 'risorse umane', 'staff'],    'struttura': ['organigramma', 'organizzazione', 'organizational chart'],    'patrimonio': ['asset', 'patrimonio netto', 'equity'],    'debiti': ['debito', 'passività', 'liabilities'],    'investimenti': ['investimento', 'spese in conto capitale'],    'rifiuti': ['rsu', 'raccolta differenziata', 'waste'],    'biblioteca': ['biblioteche', 'library', 'prestiti libri']}QUERY_TEMPLATES = {    'DELIBERE_CC': [        'site:{DOMAIN} deliberazioni consiglio comunale {YEAR}',        'site:{DOMAIN} delibere cc {YEAR} filetype:pdf',        'site:{DOMAIN} inurl:albo deliberazioni consiglio {YEAR}',        '"{COMUNE}" delibere consiglio comunale {YEAR} filetype:pdf',        'site:{DOMAIN} "delibera" AND "consiglio comunale" AND "{YEAR}"',        'site:{DOMAIN} albo pretorio deliberazioni cc {YEAR_WORD}',        '"{COMUNE}" atti consiglio comunale {YEAR}',        'site:{DOMAIN} inurl:delibere consiglio {YEAR}',        'site:{DOMAIN} "numero delibera" consiglio {YEAR} filetype:pdf',        'deliberazioni consiglio "{COMUNE}" {YEAR} site:{DOMAIN}'    ],        'DELIBERE_GC': [        'site:{DOMAIN} deliberazioni giunta comunale {YEAR}',        'site:{DOMAIN} delibere gc {YEAR} filetype:pdf',        'site:{DOMAIN} inurl:albo deliberazioni giunta {YEAR}',        '"{COMUNE}" delibere giunta comunale {YEAR} filetype:pdf',        'site:{DOMAIN} "delibera" AND "giunta comunale" AND "{YEAR}"',        'site:{DOMAIN} albo pretorio deliberazioni giunta {YEAR_WORD}',        '"{COMUNE}" atti giunta {YEAR}',        'site:{DOMAIN} inurl:delibere giunta {YEAR}',        'site:{DOMAIN} "numero delibera" giunta {YEAR} filetype:pdf',        'deliberazioni giunta "{COMUNE}" {YEAR} site:{DOMAIN}'    ],        'SEDUTE_CC': [        'site:{DOMAIN} sedute consiglio comunale {YEAR}',        'site:{DOMAIN} verbali consiglio {YEAR} filetype:pdf',        '"{COMUNE}" sedute cc {YEAR}',        'site:{DOMAIN} "ordine del giorno" consiglio {YEAR}',        'site:{DOMAIN} inurl:consiglio sedute {YEAR}',        'convocazioni consiglio comunale "{COMUNE}" {YEAR}',        'site:{DOMAIN} "consiglio comunale" riunioni {YEAR}'    ],        'SEDUTE_GC': [        'site:{DOMAIN} sedute giunta comunale {YEAR}',        'site:{DOMAIN} verbali giunta {YEAR} filetype:pdf',        '"{COMUNE}" sedute giunta {YEAR}',        'site:{DOMAIN} "ordine del giorno" giunta {YEAR}',        'site:{DOMAIN} inurl:giunta sedute {YEAR}',        'convocazioni giunta comunale "{COMUNE}" {YEAR}',        'site:{DOMAIN} "giunta comunale" riunioni {YEAR}'    ],        'PERSONALE': [        'site:{DOMAIN} personale dipendenti {YEAR}',        'site:{DOMAIN} organico comunale {YEAR} filetype:pdf',        '"{COMUNE}" dotazione organica {YEAR}',        'site:{DOMAIN} "numero dipendenti" {YEAR}',        'site:{DOMAIN} inurl:personale dotazione {YEAR}',        'risorse umane "{COMUNE}" {YEAR}',        'site:{DOMAIN} "pianta organica" {YEAR}',        'site:{DOMAIN} "conto annuale" personale {YEAR}',        'dipendenti comunali "{COMUNE}" {YEAR} site:{DOMAIN}'    ],        'STRUTTURA': [        'site:{DOMAIN} organigramma {YEAR}',        'site:{DOMAIN} struttura organizzativa {YEAR} filetype:pdf',        '"{COMUNE}" organigramma comunale {YEAR}',        'site:{DOMAIN} "organizational chart" {YEAR}',        'site:{DOMAIN} inurl:organigramma {YEAR}',        'struttura amministrativa "{COMUNE}" {YEAR}',        'site:{DOMAIN} "assetto organizzativo" {YEAR}'    ],        'SERVIZIO_CIVILE': [        'site:{DOMAIN} servizio civile {YEAR}',        '"{COMUNE}" volontari servizio civile {YEAR}',        'site:{DOMAIN} "servizio civile universale" {YEAR}',        'site:{DOMAIN} inurl:servizio-civile {YEAR}',        'bandi servizio civile "{COMUNE}" {YEAR}',        'site:{DOMAIN} progetti servizio civile {YEAR}'    ],        'PERSONALE_ETA_GENERE': [        'site:{DOMAIN} personale età genere {YEAR}',        'site:{DOMAIN} "fasce di età" dipendenti {YEAR} filetype:pdf',        '"{COMUNE}" dipendenti età sesso {YEAR}',        'site:{DOMAIN} "composizione del personale" età genere {YEAR}',        'anagrafica dipendenti "{COMUNE}" {YEAR}'    ],        'PATRIMONIO_NETTO': [        'site:{DOMAIN} patrimonio netto {YEAR}',        'site:{DOMAIN} bilancio patrimonio {YEAR} filetype:pdf',        '"{COMUNE}" patrimonio comunale {YEAR}',        'site:{DOMAIN} "stato patrimoniale" {YEAR}',        'site:{DOMAIN} inurl:bilancio patrimonio {YEAR}',        'rendiconto patrimonio netto "{COMUNE}" {YEAR}'    ],        'DEBITI': [        'site:{DOMAIN} debiti {YEAR}',        'site:{DOMAIN} esposizione debitoria {YEAR} filetype:pdf',        '"{COMUNE}" debito comunale {YEAR}',        'site:{DOMAIN} "stock di debito" {YEAR}'    ],        'RISULTATO_ECONOMICO': [        'site:{DOMAIN} risultato economico {YEAR}',        'site:{DOMAIN} conto economico {YEAR} filetype:pdf',        '"{COMUNE}" risultato di amministrazione {YEAR}'    ],        'INVESTIMENTI_MISSIONE': [        'site:{DOMAIN} investimenti missione {MISSIONE} {YEAR}',        'site:{DOMAIN} spese investimento "{MISSIONE}" {YEAR} filetype:pdf',        '"{COMUNE}" investimenti {MISSIONE} {YEAR}',        'site:{DOMAIN} "spese in conto capitale" missione {MISSIONE} {YEAR}'    ],        'SOCIAL_MEDIA': [        'site:{DOMAIN} {SOCIAL}',        '"{COMUNE}" {SOCIAL} ufficiale',        'pagina {SOCIAL} "{COMUNE}"',        '{SOCIAL}.com/{COMUNE}'    ],        'POLIZIA_LOCALE': [        'site:{DOMAIN} polizia locale {YEAR}',        'site:{DOMAIN} vigili urbani {YEAR} filetype:pdf',        '"{COMUNE}" polizia municipale organico {YEAR}',        'site:{DOMAIN} "corpo di polizia locale" {YEAR}'    ],        'ART_208_CDS': [        'site:{DOMAIN} art 208 cds {YEAR}',        'site:{DOMAIN} "articolo 208" codice della strada {YEAR} filetype:pdf',        '"{COMUNE}" proventi articolo 208 {YEAR}'    ],        'EDILIZIA': [        'site:{DOMAIN} permessi costruire {YEAR}',        'site:{DOMAIN} edilizia titoli abilitativi {YEAR} filetype:pdf',        '"{COMUNE}" concessioni edilizie {YEAR}',        'site:{DOMAIN} "pratiche edilizie" {YEAR}'    ],        'MANUTENZIONI': [        'site:{DOMAIN} manutenzioni {YEAR}',        'site:{DOMAIN} "manutenzione ordinaria" {YEAR} filetype:pdf',        '"{COMUNE}" interventi manutenzione {YEAR}'    ],        'BIBLIOTECA': [        'site:{DOMAIN} biblioteca {YEAR}',        'site:{DOMAIN} prestiti libri {YEAR} filetype:pdf',        '"{COMUNE}" servizi biblioteca {YEAR}'    ],        'RIFIUTI_RD': [        'site:{DOMAIN} raccolta differenziata {YEAR}',        'site:{DOMAIN} percentuale differenziata {YEAR} filetype:pdf',        '"{COMUNE}" rifiuti raccolta differenziata {YEAR}',        'site:{DOMAIN} "rsu" differenziata {YEAR}'    ],        'RIFIUTI_FRAZIONE': [        'site:{DOMAIN} raccolta {FRAZIONE} {YEAR}',        '"{COMUNE}" rifiuti {FRAZIONE} {YEAR} filetype:pdf'    ],        'PROGETTI_PNRR': [        'site:{DOMAIN} PNRR {YEAR}',        'site:{DOMAIN} progetti PNRR {YEAR} filetype:pdf',        '"{COMUNE}" piano nazionale ripresa resilienza {YEAR}',        'site:{DOMAIN} "fondi PNRR" {YEAR}'    ],        'PROGETTI_OPERE': [        'site:{DOMAIN} opere pubbliche {YEAR}',        'site:{DOMAIN} lavori pubblici {YEAR} filetype:pdf',        '"{COMUNE}" programma opere pubbliche {YEAR}'    ],        'GENERIC': [        'site:{DOMAIN} "{LABEL}" {YEAR}',        'site:{DOMAIN} {LABEL} {YEAR} filetype:pdf',        '"{COMUNE}" {LABEL} {YEAR}',        'site:{DOMAIN} inurl:{LABEL} {YEAR}',        '{LABEL} comune "{COMUNE}" {YEAR}'    ]}EXTERNAL_QUERY_TEMPLATES = {    'ISTAT_POPOLAZIONE': [        'site:istat.it popolazione residente "{COMUNE}" {YEAR}',        'site:dati.istat.it popolazione {YEAR} "{COMUNE}"',        'ISTAT popolazione comunale "{COMUNE}" {YEAR} filetype:xls',        'site:istat.it demo popolazione {YEAR}'    ],        'ISTAT_NATI_MORTI': [        'site:istat.it nati morti "{COMUNE}" {YEAR}',        'site:dati.istat.it movimento naturale {YEAR} "{COMUNE}"',        'ISTAT natalità mortalità "{COMUNE}" {YEAR}'    ],        'ISTAT_STRANIERI': [        'site:istat.it stranieri residenti "{COMUNE}" {YEAR}',        'site:dati.istat.it popolazione straniera {YEAR}'    ],        'ISPRA_RIFIUTI': [        'site:isprambiente.gov.it rifiuti urbani "{COMUNE}" {YEAR}',        'ISPRA rapporto rifiuti urbani {YEAR} filetype:pdf'    ],        'MEF_IMU': [        'site:finanze.gov.it IMU "{COMUNE}" {YEAR}',        'MEF gettito IMU comunale {YEAR}'    ],        'MEF_IRPEF': [        'site:finanze.gov.it IRPEF "{COMUNE}" {YEAR}',        'MEF addizionale IRPEF comunale {YEAR}'    ],        'BDAP_CONTABILI': [        'site:bdap.tesoro.it "{COMUNE}" bilancio {YEAR}',        'OpenCivitas "{COMUNE}" dati contabili {YEAR}'    ]}print("Query templates defined successfully")print(f"Internal categories: {len(QUERY_TEMPLATES)}")print(f"External categories: {len(EXTERNAL_QUERY_TEMPLATES)}")

In [ ]:
# QueryBuilder Module - Part 2: Cell Categorizationdef categorize_cell(row_label, section_context):    """    Categorize a cell based on row label and section context.    Returns category string for query template selection.    """    row_lower = row_label.lower()    section_lower = section_context.lower() if section_context else ""        # Deliberazioni    if 'deliber' in row_lower or 'deliber' in section_lower:        if 'giunta' in row_lower or 'giunta' in section_lower or 'gc' in row_lower:            return 'DELIBERE_GC'        elif 'consiglio' in row_lower or 'consiglio' in section_lower or 'cc' in row_lower:            return 'DELIBERE_CC'        return 'DELIBERE_CC'  # default        # Sedute    if 'sedut' in row_lower or 'sedut' in section_lower or 'convocaz' in row_lower:        if 'giunta' in row_lower or 'giunta' in section_lower:            return 'SEDUTE_GC'        elif 'consiglio' in row_lower or 'consiglio' in section_lower:            return 'SEDUTE_CC'        return 'SEDUTE_CC'        # Personale    if any(kw in row_lower for kw in ['dipendent', 'personale', 'organico', 'dotazione']):        if any(kw in row_lower for kw in ['età', 'genere', 'sesso', 'fasc']):            return 'PERSONALE_ETA_GENERE'        return 'PERSONALE'        # Struttura organizzativa    if any(kw in row_lower for kw in ['organigramma', 'struttura', 'organizzativ']):        return 'STRUTTURA'        # Servizio civile    if 'servizio civile' in row_lower or 'volontar' in row_lower:        return 'SERVIZIO_CIVILE'        # Patrimonio    if 'patrimonio' in row_lower:        return 'PATRIMONIO_NETTO'        # Debiti    if 'debit' in row_lower or 'passività' in row_lower:        return 'DEBITI'        # Risultato economico    if 'risultato' in row_lower and ('economic' in row_lower or 'amministraz' in row_lower):        return 'RISULTATO_ECONOMICO'        # Investimenti    if 'investiment' in row_lower or 'spese in conto capitale' in row_lower:        # Check for mission        if any(kw in row_lower for kw in ['mission', 'settore', 'area']):            return 'INVESTIMENTI_MISSIONE'        return 'INVESTIMENTI_MISSIONE'        # Social media    if any(kw in row_lower for kw in ['facebook', 'twitter', 'instagram', 'youtube', 'linkedin', 'social']):        return 'SOCIAL_MEDIA'        # Polizia locale    if any(kw in row_lower for kw in ['polizia', 'vigili', 'municipal']):        return 'POLIZIA_LOCALE'        # Art. 208 CDS    if '208' in row_lower or ('art' in row_lower and 'strada' in row_lower):        return 'ART_208_CDS'        # Edilizia    if any(kw in row_lower for kw in ['edili', 'permess', 'concessio', 'costrui']):        return 'EDILIZIA'        # Manutenzioni    if 'manutenz' in row_lower:        return 'MANUTENZIONI'        # Biblioteca    if 'bibliote' in row_lower or 'prestit' in row_lower:        return 'BIBLIOTECA'        # Rifiuti    if 'rifiut' in row_lower or 'rsu' in row_lower or 'raccolta' in row_lower:        if 'differenziat' in row_lower:            return 'RIFIUTI_RD'        # Check for specific fraction        if any(kw in row_lower for kw in ['organic', 'carta', 'plastica', 'vetro', 'secco']):            return 'RIFIUTI_FRAZIONE'        return 'RIFIUTI_RD'        # PNRR    if 'pnrr' in row_lower or 'ripresa' in row_lower:        return 'PROGETTI_PNRR'        # Opere pubbliche    if any(kw in row_lower for kw in ['opere', 'lavori', 'pubbl']):        return 'PROGETTI_OPERE'        # Default    return 'GENERIC'# Test categorizationtest_cases = [    ("Deliberazioni Consiglio Comunale", None),    ("Delibere Giunta", None),    ("Personale dipendenti", None),    ("Raccolta differenziata %", None),    ("Facebook", None)]print("Testing categorize_cell function:")for label, section in test_cases:    category = categorize_cell(label, section)    print(f"  '{label}' -> {category}")print("\nCategorization function ready")

In [ ]:
# QueryBuilder Module - Part 3: Query Generationdef build_queries(category, domain, comune, year, extra_params=None):    """    Build search queries for a given category.        Args:        category: Category string (e.g., 'DELIBERE_CC')        domain: Website domain        comune: Municipality name        year: Year (int)        extra_params: Dict with optional params like 'MISSIONE', 'SOCIAL', 'FRAZIONE', 'LABEL'        Returns:        List of dicts with 'query' and 'priority' keys    """    extra_params = extra_params or {}        # Get templates    templates = []        # Check if category is external    if category in EXTERNAL_QUERY_TEMPLATES:        if not ALLOW_EXTERNAL_OFFICIAL:            return []  # Don't use external sources        templates = EXTERNAL_QUERY_TEMPLATES[category]    else:        templates = QUERY_TEMPLATES.get(category, QUERY_TEMPLATES['GENERIC'])        # Year word mapping    year_words = {        2020: 'duemilaventi',        2021: 'duemilaventuno',        2022: 'duemilaventidue',        2023: 'duemilaventitré',        2024: 'duemilaventiquattro',        2025: 'duemilaventicinque'    }    year_word = year_words.get(year, str(year))        queries = []    for template in templates:        query = template.replace('{DOMAIN}', domain)        query = query.replace('{COMUNE}', comune)        query = query.replace('{YEAR}', str(year))        query = query.replace('{YEAR_WORD}', year_word)                # Replace extra parameters        if '{MISSIONE}' in query:            missione = extra_params.get('MISSIONE', 'servizi generali')            query = query.replace('{MISSIONE}', missione)                if '{SOCIAL}' in query:            social = extra_params.get('SOCIAL', 'facebook')            query = query.replace('{SOCIAL}', social)                if '{FRAZIONE}' in query:            frazione = extra_params.get('FRAZIONE', 'organico')            query = query.replace('{FRAZIONE}', frazione)                if '{LABEL}' in query:            label = extra_params.get('LABEL', '')            query = query.replace('{LABEL}', label)                # Calculate priority (1-10)        priority = 5  # base                if 'filetype:pdf' in query:            priority += 2                if 'site:' + domain in query:            priority += 1                if 'inurl:' in query:            priority += 1                if '"' in query:  # has quoted terms            priority += 1                priority = min(10, priority)                queries.append({            'query': query,            'priority': priority        })        # Sort by priority (descending)    queries.sort(key=lambda x: x['priority'], reverse=True)        return queries# Test query buildingtest_category = 'DELIBERE_CC'test_queries = build_queries(test_category, DOMAIN, nome_comune, 2023)print(f"Generated {len(test_queries)} queries for category '{test_category}':")for i, q in enumerate(test_queries[:5], 1):    print(f"  {i}. [Priority {q['priority']}] {q['query']}")print(f"\nQuery builder ready (will generate {len(test_queries)} queries per cell)")

In [ ]:
# Web Crawling Module - Part 1: Website Crawlerdef crawl_website(base_url, domain, max_pages=500):    """    Crawl website to find HTML pages and PDF documents.        Returns:        Dict with 'html_docs' and 'pdf_docs' lists    """    print(f"Starting crawl of {base_url}...")        visited = set()    to_visit = [base_url]    html_docs = []    pdf_docs = []    pdf_dir = '/tmp/pdfs/'        os.makedirs(pdf_dir, exist_ok=True)        headers = {        'User-Agent': 'Mozilla/5.0 (compatible; DataExtractor/1.0)'    }        # Try to get robots.txt and sitemap.xml    try:        robots_url = urljoin(base_url, '/robots.txt')        resp = requests.get(robots_url, headers=headers, timeout=10)        if resp.status_code == 200:            print(f"  Found robots.txt")            # Look for sitemap            for line in resp.text.split('\n'):                if 'sitemap' in line.lower() and 'http' in line:                    sitemap_url = line.split()[-1]                    print(f"  Found sitemap: {sitemap_url}")    except Exception as e:        pass        # Crawl pages    pbar = tqdm(total=max_pages, desc="Crawling")        while to_visit and len(visited) < max_pages:        url = to_visit.pop(0)                if url in visited:            continue                visited.add(url)        pbar.update(1)                try:            # Skip non-same-domain URLs            if domain not in urlparse(url).netloc:                continue                        # Handle PDFs            if url.lower().endswith('.pdf'):                try:                    pdf_resp = requests.get(url, headers=headers, timeout=30)                    if pdf_resp.status_code == 200:                        pdf_filename = os.path.join(pdf_dir, f"doc_{len(pdf_docs)}.pdf")                        with open(pdf_filename, 'wb') as f:                            f.write(pdf_resp.content)                        pdf_docs.append({                            'url': url,                            'path': pdf_filename,                            'type': 'pdf'                        })                except Exception as e:                    pass                continue                        # Get HTML page            resp = requests.get(url, headers=headers, timeout=15)            if resp.status_code != 200:                continue                        soup = BeautifulSoup(resp.content, 'html.parser')                        # Store HTML doc            html_docs.append({                'url': url,                'html': resp.text,                'type': 'html'            })                        # Find links            for link in soup.find_all('a', href=True):                href = link['href']                full_url = urljoin(url, href)                                # Add to queue                if full_url not in visited and full_url not in to_visit:                    to_visit.append(full_url)                        time.sleep(0.5)  # Be polite                    except Exception as e:            pass        pbar.close()        print(f"\nCrawl complete:")    print(f"  HTML pages: {len(html_docs)}")    print(f"  PDF documents: {len(pdf_docs)}")        return {        'html_docs': html_docs,        'pdf_docs': pdf_docs    }print("Website crawler ready")

In [ ]:
# Web Crawling Module - Part 2: PDF Text Extractiondef extract_text_from_pdf(pdf_path):    """    Extract text from PDF using pdfplumber.        Returns:        Text string    """    try:        text = ""        with pdfplumber.open(pdf_path) as pdf:            for page in pdf.pages:                page_text = page.extract_text()                if page_text:                    text += page_text + "\n"        return text    except Exception as e:        print(f"  Error extracting PDF {pdf_path}: {e}")        return ""print("PDF extraction function ready")

In [ ]:
# Web Crawling Module - Part 3: HTML Text Extractiondef extract_text_from_html(url, html_content=None):    """    Extract clean text from HTML using trafilatura.    Falls back to BeautifulSoup if trafilatura fails.        Returns:        Text string    """    try:        # If HTML content provided, use it        if html_content:            text = trafilatura.extract(html_content)            if text:                return text                # Otherwise download        downloaded = trafilatura.fetch_url(url)        if downloaded:            text = trafilatura.extract(downloaded)            if text:                return text                # Fallback to BeautifulSoup        if html_content:            soup = BeautifulSoup(html_content, 'html.parser')        else:            resp = requests.get(url, timeout=15)            soup = BeautifulSoup(resp.content, 'html.parser')                # Remove script and style elements        for script in soup(["script", "style"]):            script.decompose()                text = soup.get_text()        lines = (line.strip() for line in text.splitlines())        chunks = (phrase.strip() for line in lines for phrase in line.split("  "))        text = '\n'.join(chunk for chunk in chunks if chunk)                return text        except Exception as e:        return ""print("HTML extraction function ready")

In [ ]:
# TF-IDF Module - Part 1: Index Buildingdef build_tfidf_index(documents):    """    Build TF-IDF index from documents.        Args:        documents: List of dicts with 'url', 'text', 'type'        Returns:        (vectorizer, tfidf_matrix)    """    print(f"Building TF-IDF index from {len(documents)} documents...")        if not documents:        print("  Warning: No documents to index")        return None, None        # Extract texts    texts = [doc.get('text', '') for doc in documents]        # Create vectorizer    vectorizer = TfidfVectorizer(        max_features=5000,        ngram_range=(1, 3),        stop_words=None,  # Keep Italian words        min_df=1,        max_df=0.8    )        try:        tfidf_matrix = vectorizer.fit_transform(texts)        print(f"  TF-IDF matrix shape: {tfidf_matrix.shape}")        print(f"  Vocabulary size: {len(vectorizer.vocabulary_)}")        return vectorizer, tfidf_matrix    except Exception as e:        print(f"  Error building index: {e}")        return None, Noneprint("TF-IDF indexing function ready")

In [ ]:
# TF-IDF Module - Part 2: Document Searchdef search_documents(query_text, vectorizer, tfidf_matrix, documents, top_k=10):    """    Search documents using TF-IDF similarity.        Returns:        List of top_k documents with scores    """    if vectorizer is None or tfidf_matrix is None:        return []        # Clean query (remove search operators)    clean_query = query_text    clean_query = re.sub(r'site:[\S]+', '', clean_query)    clean_query = re.sub(r'filetype:[\S]+', '', clean_query)    clean_query = re.sub(r'inurl:[\S]+', '', clean_query)    clean_query = re.sub(r'[\"\"\']', '', clean_query)  # Remove quotes    clean_query = clean_query.replace(' AND ', ' ').replace(' OR ', ' ')    clean_query = clean_query.strip()        if not clean_query:        return []        try:        # Vectorize query        query_vec = vectorizer.transform([clean_query])                # Compute similarities        similarities = cosine_similarity(query_vec, tfidf_matrix).flatten()                # Get top k indices        top_indices = similarities.argsort()[-top_k:][::-1]                # Build results        results = []        for idx in top_indices:            if similarities[idx] > 0:  # Only positive similarity                results.append({                    'document': documents[idx],                    'score': float(similarities[idx]),                    'rank': len(results) + 1                })                return results        except Exception as e:        return []print("Document search function ready")

In [ ]:
# CSV Processing Module - Part 1: CSV Loadingdef load_csv_robust(csv_path):    """    Load CSV and detect structure.        Returns:        (df, year_columns, sections_map)    """    print(f"Loading CSV: {os.path.basename(csv_path)}")        try:        df = pd.read_csv(csv_path, encoding='utf-8')    except:        try:            df = pd.read_csv(csv_path, encoding='latin-1')        except Exception as e:            print(f"  Error loading CSV: {e}")            return None, [], {}        # Detect year columns    year_pattern = re.compile(r'\b20\d{2}\b')    year_columns = []        for col in df.columns:        if year_pattern.search(str(col)):            year_columns.append(col)        print(f"  Detected {len(year_columns)} year columns: {year_columns}")        # Detect section headers (rows with few filled cells)    sections_map = {}    current_section = None        for idx, row in df.iterrows():        # Count non-null values        non_null = row.notna().sum()        total = len(row)                # If less than 30% filled, likely a section header        if non_null / total < 0.3 and non_null > 0:            # Get first non-null value as section name            for val in row:                if pd.notna(val) and str(val).strip():                    current_section = str(val).strip()                    print(f"  Section header at row {idx}: {current_section}")                    break        else:            sections_map[idx] = current_section        return df, year_columns, sections_mapprint("CSV loading function ready")

In [ ]:
# CSV Processing Module - Part 2: Missing Cell Detectiondef is_missing_cell(value):    """    Check if a cell value is missing or needs to be filled.        Returns:        True if missing    """    if pd.isna(value):        return True        value_str = str(value).strip().lower()        if not value_str:        return True        # Check for placeholder text    placeholders = ['inserire', 'inserisci', '…', '...', 'n/d', 'n.d.', 'nd', 'da compilare']        for placeholder in placeholders:        if placeholder in value_str:            return True        return False# Testtest_values = [None, "", "Inserire dato", "123", "…", "45.5"]print("Testing is_missing_cell:")for val in test_values:    print(f"  '{val}' -> {is_missing_cell(val)}")print("\nMissing cell detection ready")

In [ ]:
# CSV Processing Module - Part 3: Missing Cells Detectiondef detect_missing_cells(df, year_columns, sections_map):    """    Detect all missing cells in year columns.        Returns:        List of (row_idx, col_name, row_label, section_context)    """    missing_cells = []        for idx, row in df.iterrows():        # Get row label (usually first column)        row_label = str(row.iloc[0]) if len(row) > 0 else f"Row {idx}"                # Get section context        section_context = sections_map.get(idx, None)                # Check year columns        for col in year_columns:            if col in df.columns:                value = row[col]                if is_missing_cell(value):                    missing_cells.append({                        'row_idx': idx,                        'col_name': col,                        'row_label': row_label,                        'section_context': section_context                    })        return missing_cellsprint("Missing cells detection function ready")

In [ ]:
# Value Extraction Moduledef extract_value_from_text(text, row_label, year):    """    Extract numeric value from text based on row label and year context.        Returns:        (value, snippet) or (None, None)    """    if not text:        return None, None        # Create search keywords from row label    keywords = row_label.lower().split()    keywords = [k for k in keywords if len(k) > 3]  # Filter short words        # Find relevant sections in text    text_lower = text.lower()    year_str = str(year)        # Look for year mentions    year_positions = [m.start() for m in re.finditer(year_str, text)]        best_value = None    best_snippet = None    best_score = 0        # Search in windows around year mentions    for pos in year_positions:        # Get context window (500 chars before and after)        start = max(0, pos - 500)        end = min(len(text), pos + 500)        window = text[start:end]        window_lower = window.lower()                # Check if keywords are in window        keyword_count = sum(1 for kw in keywords if kw in window_lower)                if keyword_count == 0:            continue                # Extract numbers from window        # Patterns: integers, decimals, percentages, currency        patterns = [            (r'€\s*([0-9]{1,3}(?:\.[0-9]{3})*(?:,[0-9]+)?)', 'currency'),            (r'([0-9]{1,3}(?:\.[0-9]{3})*(?:,[0-9]+)?)\s*€', 'currency'),            (r'([0-9]+(?:,[0-9]+)?)\s*%', 'percentage'),            (r'([0-9]{1,3}(?:\.[0-9]{3})*(?:,[0-9]+)?)\s*(?:kg|tonnellate|ton)', 'weight'),            (r'([0-9]{1,3}(?:\.[0-9]{3})*(?:,[0-9]+)?)', 'number')        ]                for pattern, value_type in patterns:            matches = re.finditer(pattern, window, re.IGNORECASE)            for match in matches:                value_str = match.group(1)                # Convert Italian number format to float                try:                    value_float = float(value_str.replace('.', '').replace(',', '.'))                                        # Calculate score based on keyword proximity                    match_pos = match.start()                    min_distance = min(abs(match_pos - window_lower.find(kw))                                      for kw in keywords if kw in window_lower)                                        score = keyword_count * 10 - min(min_distance / 10, 50)                                        if score > best_score:                        best_score = score                        best_value = value_str                        # Get snippet around match                        snippet_start = max(0, match.start() - 100)                        snippet_end = min(len(window), match.end() + 100)                        best_snippet = window[snippet_start:snippet_end].strip()                        best_snippet = best_snippet.replace('\n', ' ')                                except:                    continue        if best_value and best_score > 5:        return best_value, best_snippet        return None, None# Test extractiontest_text = "Nel 2023 il comune ha registrato 150 dipendenti, con un incremento del 5% rispetto al 2022."test_label = "Numero dipendenti"value, snippet = extract_value_from_text(test_text, test_label, 2023)print(f"Test extraction: value='{value}', snippet='{snippet[:80]}...'" if snippet else "None")print("\nValue extraction function ready")

In [ ]:
# Main Pipeline - Part 1: Process Single CSVdef process_single_csv(csv_path, vectorizer, tfidf_matrix, documents):    """    Process a single CSV file to fill missing values.        Returns:        (filled_df, sources_list, queries_audit)    """    print(f"\n{'='*60}")    print(f"Processing: {os.path.basename(csv_path)}")    print('='*60)        # Load CSV    df, year_columns, sections_map = load_csv_robust(csv_path)        if df is None:        return None, [], []        # Filter year columns to only YEARS_TO_FILL    year_columns_filtered = []    for col in year_columns:        # Extract year from column name        year_match = re.search(r'\b(20\d{2})\b', str(col))        if year_match:            year_int = int(year_match.group(1))            if year_int in YEARS_TO_FILL:                year_columns_filtered.append(col)        print(f"Columns to process: {year_columns_filtered}")        # Detect missing cells    missing_cells = detect_missing_cells(df, year_columns_filtered, sections_map)    print(f"Found {len(missing_cells)} missing cells")        if not missing_cells:        print("No missing cells to fill")        return df, [], []        # Process each missing cell    sources_list = []    queries_audit = []    filled_count = 0        for cell_info in tqdm(missing_cells, desc="Filling cells"):        row_idx = cell_info['row_idx']        col_name = cell_info['col_name']        row_label = cell_info['row_label']        section_context = cell_info['section_context']                # Extract year from column name        year_match = re.search(r'\b(20\d{2})\b', str(col_name))        if not year_match:            continue        year = int(year_match.group(1))                # Categorize cell        category = categorize_cell(row_label, section_context)                # Build queries        extra_params = {'LABEL': row_label}        queries = build_queries(category, DOMAIN, nome_comune, year, extra_params)                # Save to audit        for q in queries:            queries_audit.append({                'input_file': os.path.basename(csv_path),                'section': section_context or 'N/A',                'row_label': row_label,                'col_year': col_name,                'query': q['query'],                'priority': q['priority'],                'notes': f"Category: {category}"            })                # Search documents with top queries        value_found = None        source_url = None        snippet = None                for q in queries[:10]:  # Try top 10 queries            results = search_documents(q['query'], vectorizer, tfidf_matrix, documents, top_k=5)                        # Try to extract value from top results            for result in results:                doc = result['document']                text = doc.get('text', '')                                extracted_value, extracted_snippet = extract_value_from_text(text, row_label, year)                                if extracted_value:                    value_found = extracted_value                    source_url = doc.get('url', '')                    snippet = extracted_snippet                    break                        if value_found:                break                # Fill cell if value found        if value_found:            df.at[row_idx, col_name] = value_found            filled_count += 1                        # Save source info            sources_list.append({                'input_file': os.path.basename(csv_path),                'row_label': row_label,                'col_year': col_name,                'value_filled': value_found,                'source_url': source_url,                'snippet': snippet            })        print(f"\nFilled {filled_count}/{len(missing_cells)} cells ({filled_count/len(missing_cells)*100:.1f}%)")        return df, sources_list, queries_auditprint("CSV processing pipeline ready")

In [ ]:
# Main Pipeline - Part 2: Execute Pipelineprint("\n" + "="*60)print("STARTING MAIN PIPELINE")print("="*60 + "\n")# Step 1: Crawl websiteprint("STEP 1: Crawling website...")crawl_results = crawl_website(base_url, DOMAIN, max_pages=500)html_docs = crawl_results['html_docs']pdf_docs = crawl_results['pdf_docs']# Step 2: Extract text from all documentsprint("\nSTEP 2: Extracting text from documents...")all_documents = []# Process HTML docsfor doc in tqdm(html_docs, desc="Processing HTML"):    text = extract_text_from_html(doc['url'], doc.get('html'))    if text:        all_documents.append({            'url': doc['url'],            'text': text,            'type': 'html'        })# Process PDF docsfor doc in tqdm(pdf_docs, desc="Processing PDFs"):    text = extract_text_from_pdf(doc['path'])    if text:        all_documents.append({            'url': doc['url'],            'text': text,            'type': 'pdf'        })print(f"\nTotal documents with text: {len(all_documents)}")# Step 3: Build TF-IDF indexprint("\nSTEP 3: Building TF-IDF index...")vectorizer, tfidf_matrix = build_tfidf_index(all_documents)if vectorizer is None:    print("ERROR: Could not build index. Exiting.")else:    # Step 4: Get CSV files    print("\nSTEP 4: Finding CSV files...")    csv_files = []        if os.path.exists(INPUT_DIR):        for file in os.listdir(INPUT_DIR):            if file.endswith('.csv'):                csv_files.append(os.path.join(INPUT_DIR, file))        print(f"Found {len(csv_files)} CSV files")        if not csv_files:        print("WARNING: No CSV files found in input directory")    else:        # Step 5: Process each CSV        print("\nSTEP 5: Processing CSV files...")                all_sources = []        all_queries = []                for csv_path in csv_files:            filled_df, sources, queries = process_single_csv(                csv_path, vectorizer, tfidf_matrix, all_documents            )                        if filled_df is not None:                # Save filled CSV                output_filename = os.path.basename(csv_path).replace('.csv', '_filled.csv')                output_path = os.path.join(OUTPUT_DIR, output_filename)                filled_df.to_csv(output_path, index=False, encoding='utf-8')                print(f"Saved: {output_filename}")                                all_sources.extend(sources)                all_queries.extend(queries)                # Step 6: Save consolidated outputs (next cell)        print("\n" + "="*60)        print("CSV processing complete")        print("="*60)

In [ ]:
# Main Pipeline - Part 3: Save Outputsprint("\nSTEP 6: Saving consolidated outputs...")# Save sources_long.csvif all_sources:    sources_df = pd.DataFrame(all_sources)    sources_path = os.path.join(OUTPUT_DIR, 'sources_long.csv')    sources_df.to_csv(sources_path, index=False, encoding='utf-8')    print(f"Saved sources_long.csv ({len(all_sources)} entries)")else:    print("No sources to save")# Save queries_generated.csvif all_queries:    queries_df = pd.DataFrame(all_queries)    queries_path = os.path.join(OUTPUT_DIR, 'queries_generated.csv')    queries_df.to_csv(queries_path, index=False, encoding='utf-8')    print(f"Saved queries_generated.csv ({len(all_queries)} entries)")else:    print("No queries to save")# Create run reportreport_path = os.path.join(OUTPUT_DIR, 'run_report.md')with open(report_path, 'w', encoding='utf-8') as f:    f.write("# Estrazione Dati Comune - Run Report\n\n")    f.write(f"**Date**: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n\n")    f.write(f"**Municipality**: {nome_comune}\n\n")    f.write(f"**Domain**: {DOMAIN}\n\n")    f.write(f"**Years**: {YEARS_TO_FILL}\n\n")        f.write("## Statistics\n\n")    f.write(f"- Documents crawled: {len(all_documents)}\n")    f.write(f"- HTML pages: {len([d for d in all_documents if d['type'] == 'html'])}\n")    f.write(f"- PDF documents: {len([d for d in all_documents if d['type'] == 'pdf'])}\n")    f.write(f"- CSV files processed: {len(csv_files)}\n")    f.write(f"- Total queries generated: {len(all_queries)}\n")    f.write(f"- Values filled: {len(all_sources)}\n\n")        f.write("## Coverage\n\n")    if all_queries:        total_cells = len(set((q['input_file'], q['row_label'], q['col_year']) for q in all_queries))        filled_cells = len(all_sources)        coverage = (filled_cells / total_cells * 100) if total_cells > 0 else 0        f.write(f"- Total missing cells: {total_cells}\n")        f.write(f"- Filled cells: {filled_cells}\n")        f.write(f"- Coverage: {coverage:.1f}%\n\n")        f.write("## Not Found Items\n\n")    if all_queries:        # Find cells with queries but no values        queried_cells = set((q['input_file'], q['row_label'], q['col_year']) for q in all_queries)        filled_cells_set = set((s['input_file'], s['row_label'], s['col_year']) for s in all_sources)        not_found = queried_cells - filled_cells_set                if not_found:            f.write(f"Found {len(not_found)} cells without values:\n\n")            for file, label, year in sorted(not_found)[:50]:  # Limit to 50                f.write(f"- {file} | {label} | {year}\n")                        if len(not_found) > 50:                f.write(f"\n... and {len(not_found) - 50} more\n")        else:            f.write("All queried cells were filled successfully!\n")        f.write("\n## Output Files\n\n")    f.write("- `*_filled.csv`: Filled CSV files\n")    f.write("- `sources_long.csv`: Source URLs and snippets for each filled value\n")    f.write("- `queries_generated.csv`: All generated queries with priorities\n")    f.write("- `run_report.md`: This report\n")print(f"Saved run_report.md")print("\n" + "="*60)print("PIPELINE COMPLETE")print("="*60)print(f"\nOutputs saved to: {OUTPUT_DIR}")